In [2]:
import tensorflow as tf
import numpy as np
from moviepy.editor import *
from pydub import AudioSegment

video = VideoFileClip('/home/andrea/Scrivania/test_video.mp4')
audio = video.audio
duration = video.duration


In [21]:
chunks = []
step = 14716*3
half_step = 7358
for t in range(int(audio.buffersize/step)):
    t1 = t*step
    t2 = (t+1)*step
    if t2 > audio.buffersize:
        chunks.append(audio.reader.buffer[t1:,0])
    else:
        chunks.append(audio.reader.buffer[t1:t2,0])

In [25]:
audio_buf = audio.reader.buffer[:,0]
r = audio_buf.shape
start = r[0] - 128000
audio_buf = audio_buf[start:]
data = []
data.append(audio_buf)

In [26]:
test_video = tf.data.Dataset.from_tensor_slices(data)

In [19]:
def get_spectrogram(waveform):
  # Padding for files with less than 16000 samples
  zero_padding = tf.zeros([128000] - tf.shape(waveform), dtype=tf.float32)

  # Concatenate audio with padding so that all audio clips will be of the
  # same length
  waveform = tf.cast(waveform, tf.float32)
  equal_length = tf.concat([waveform, zero_padding], 0)
  spectrogram = tf.signal.stft(
      equal_length, frame_length=255, frame_step=128)

  spectrogram = tf.abs(spectrogram)

  return tf.expand_dims(spectrogram, -1)

In [27]:
spectrogram_ds = test_video.map(get_spectrogram, num_parallel_calls=-1)
batch_size = 1
test_ds = spectrogram_ds.batch(batch_size)
test_ds = test_ds.cache().prefetch(-1)

NOTE:

prendere solo l'audio dello spezzone corretto porta comunque a una misclassificazione

In [ ]:
model = tf.keras.models.load_model("audio_model.h5")

In [28]:
model.predict_classes(test_ds)

array([0])

In [13]:
newAudio = AudioSegment(data=chunks[0])

AttributeError: 'numpy.ndarray' object has no attribute 'read'